# Tutorial 4 - Setting parameter values

在教程 1 和教程 2 中，我们看到了如何使用所有默认设置运行 PyBaMM 模型。 但是，PyBaMM 还允许您为您的应用程序调整这些设置。 在本教程中，我们将了解如何更改 PyBaMM 中的参数。

In [20]:
import pybamm
import os
os.chdir(pybamm.__path__[0]+'/..')

## 更改整个参数集

PyBaMM 有许多内置的参数集（在这里查看列表），可以选择做

In [21]:
chemistry = pybamm.parameter_sets.Chen2020

这个变量是一个字典，每个组件都有对应的参数子集。

In [22]:
chemistry

{'chemistry': 'lithium_ion',
 'cell': 'LGM50_Chen2020',
 'negative electrode': 'graphite_Chen2020',
 'separator': 'separator_Chen2020',
 'positive electrode': 'nmc_Chen2020',
 'electrolyte': 'lipf6_Nyman2008',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example',
 'citation': 'Chen2020'}

可以在此处找到有关每个子集的更多详细信息。

现在我们可以将“chemistry”传递给 ParameterValues 以创建参数值字典

In [23]:
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

我们可以看到字典中存储的所有参数

In [24]:
parameter_values

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell volume [m3]': 2.42e-05,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x00000266A0DFD820>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x00000266A0DFD700>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initial concentration in positive electrode [mol.m-3]': 17038.0,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 'Initial temperature [K]': 298.15,
 'Inner SEI electron conductivity [S.m-1]': 8.95e-14,
 'Inn

或者我们可以搜索一个特定的参数

In [25]:
parameter_values.search("Current")

Current function [A]	5.0
Negative current collector conductivity [S.m-1]	58411000.0
Negative current collector density [kg.m-3]	8960.0
Negative current collector specific heat capacity [J.kg-1.K-1]	385.0
Negative current collector thermal conductivity [W.m-1.K-1]	401.0
Negative current collector thickness [m]	1.2e-05
Negative electrode exchange-current density [A.m-2]	<function graphite_LGM50_electrolyte_exchange_current_density_Chen2020 at 0x00000266A0DFD430>
Positive current collector conductivity [S.m-1]	36914000.0
Positive current collector density [kg.m-3]	2700.0
Positive current collector specific heat capacity [J.kg-1.K-1]	897.0
Positive current collector thermal conductivity [W.m-1.K-1]	237.0
Positive current collector thickness [m]	1.6e-05
Positive electrode exchange-current density [A.m-2]	<function nmc_LGM50_electrolyte_exchange_current_density_Chen2020 at 0x00000266A0DFD0D0>
Ratio of inner and outer SEI exchange current densities	1.0
SEI reaction exchange current density [A

要使用此参数集运行模拟，我们可以照常进行，但将参数作为关键字参数传递

In [26]:
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=3554.1817016732207, step=35.54181701673221),…

## Change individual parameters

我们经常希望快速更改少量参数值来调查行为或电池如何变化。 在这种情况下，我们可以更改参数值，而无需离开您正在使用的笔记本或脚本。

我们开始初始化模型和参数值

In [27]:
model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Chen2020)

在本例中，我们将电流更改为 10 A

In [28]:
parameter_values["Current function [A]"] = 50
parameter_values["Nominal cell capacity [A.h]"] =3.6

现在我们只需要使用新的参数值运行模拟

In [29]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
output_variables = ["Terminal voltage [V]", "Current [A]"]

sim.solve([0, 10])
sim.plot(output_variables=output_variables)

interactive(children=(FloatSlider(value=0.0, description='t', max=9.751368631086645, step=0.09751368631086646)…

## 驱动循环

您可以导入数据集并创建插值以作为当前函数传递，从而实现行驶周期。

In [30]:
import pandas as pd    # needed to read the csv data file

# Import drive cycle from file
drive_cycle = pd.read_csv("pybamm/input/drive_cycles/UDDS.csv", comment="#", header=None).to_numpy()

# Create interpolant
timescale = parameter_values.evaluate(model.timescale)
current_interpolant = pybamm.Interpolant(drive_cycle[:, 0], drive_cycle[:, 1], timescale * pybamm.t)

# Set drive cycle
parameter_values["Current function [A]"] = current_interpolant

请注意，您的驾驶循环数据可以存储在任何地方，您只需要传递文件的路径即可。 然后，再次，模型可以像往常一样求解，但请注意，现在，如果未指定 t_eval，求解器将从数据集中获取时间点。

In [31]:
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve()
sim.plot(["Current [A]", "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=1016.8210751962804, step=10.168210751962803)…

## 自定义电流函数

或者，我们可以将电流定义为时间的任意函数

In [32]:
import numpy as np

def my_current(t):
    return pybamm.sin(2 * np.pi * t / 60)+5

parameter_values["Current function [A]"] = my_current

我们现在可以再次求解模型。 在这种情况下，我们可以将 t_eval 传递给求解器，以确保我们有足够的时间点来解析输出中的函数。

In [33]:
model = pybamm.lithium_ion.SPMe()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
t_eval = np.arange(0, 121, 1)
sim.solve([0,3600])
sim.plot(["Current [A]", "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=3549.015284479444, step=35.49015284479444), …

In [34]:
parameter_values.search("initial")

EC initial concentration in electrolyte [mol.m-3]	4541.0
Initial concentration in electrolyte [mol.m-3]	1000.0
Initial concentration in negative electrode [mol.m-3]	29866.0
Initial concentration in positive electrode [mol.m-3]	17038.0
Initial inner SEI thickness [m]	2.5e-09
Initial outer SEI thickness [m]	2.5e-09
Initial temperature [K]	298.15


在这个笔记本中，我们看到了如何更改模型的参数。 在教程 5 中，我们展示了如何定义和运行实验。